# HMM Analysis for Andreev State Spectroscopy with Clearing Tones

This notebook provides functionality for analyzing Andreev state spectroscopy data
using Hidden Markov Models (HMM) with clearing tones.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import quasiparticleFunctions as qp
from hmm_utils import (create_physics_based_transition_matrix,
                          get_means_covars)
from hmm_analysis_with_clearing import HMMAnalyzerWithClearing
from hmm_workflows_with_clearing import *
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle
import pandas as pd
import seaborn as sns

## Single Power at Fixed Flux Workflow with Clearing Tone

In [ ]:
# Initialize analyzer
data_dir = "/path/to/your/data/"
analyzer = HMMAnalyzerWithClearing(data_dir, num_modes=2)

In [ ]:
# Load data files
analyzer.load_data_files("phi_0p450")

In [ ]:
# Process data for specific parameters
atten = 26
clearing_freq = 5.0  # GHz
clearing_power = -10.0  # dBm
analyzer.load_and_process_data(atten=atten, clearing_freq=clearing_freq, clearing_power=clearing_power)

In [ ]:
# Get initial parameters
result = get_means_covars(analyzer.data, analyzer.num_modes)
means_guess = result['means']
covars_guess = result['covariances']

In [ ]:
# Initialize and fit model
analyzer.initialize_model(means_guess, covars_guess)
analyzer.fit_model()

In [ ]:
# Decode states and calculate probabilities
logprob, states = analyzer.decode_states()
mean_occ, probs = analyzer.calculate_occupation_probabilities(states)
print(f"Mean occupation: {mean_occ}")
print(f"Probabilities: {probs}")

In [ ]:
# Calculate SNRs
snrs = analyzer.calculate_snrs()
print("SNRs:", snrs)

In [ ]:
# Save all results and plots
analyzer.save_analysis_results(states, atten, means_guess, covars_guess)

## Variable Power at Fixed Flux (Bootstrapping) Workflow with Clearing Tone

In [ ]:
# Run bootstrapping analysis with clearing tone parameters
bootstrapping_analysis_with_clearing(
    analyzer,
    "phi_0p450",
    non_linear_atten=26,
    clearing_freq=5.0,
    clearing_power=-10.0,
    snr_threshold=1.5
)

## Variable Power and Flux (Bootstrapping) Workflow with Clearing Tone

In [ ]:
# Run flux sweep analysis with clearing tone parameters
analyze_flux_sweep_with_clearing(
    data_dir,
    num_modes=2,
    non_linear_atten=26,
    clearing_freq=5.0,
    clearing_power=-10.0,
    snr_threshold=1.5
)